In [1]:
import pandas as pd
import pickle

### Same logic as Calculating_Distances.ipynb
* Calculating neighbor info for compliance operations performed by NYC
* See associated markdown for info

In [2]:
with open("df_address.pkl", 'rb') as picklefile: 
    df4 = pickle.load(picklefile)

In [3]:
df = df4[['Block','XCoord','YCoord']].copy()
df['x'] = 1

In [4]:
df = df.groupby(['Block','XCoord','YCoord'], as_index=False).x.count()
del df['x']

In [5]:
df = df.groupby('Block')['XCoord','YCoord'].mean()

In [6]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(df.values, metric='euclidean')
dist_matrix = squareform(distances)

In [7]:
indicies = ['block' + str(i) for i in df.index.tolist()]

In [8]:
dist_matrix = pd.DataFrame(dist_matrix, columns = indicies, index = df.index)

In [9]:
smallest = 6

m_len = len(dist_matrix.columns)
for col in dist_matrix.columns[:m_len]:
    l = dist_matrix[col].nsmallest(smallest).index
    for i in range(len(l)):
        col_2c = m_len + i
        row_2c = int(col[5:])
        dist_matrix.loc[row_2c, col_2c] = int(l[i])
        
k = 0
for j in range(m_len, m_len + smallest):
    k += 1
    dist_matrix.rename(columns={j: 'closest_' + str(k-1)}, inplace=True)

In [10]:
df_closest = dist_matrix.iloc[:,-smallest:].reset_index()

In [11]:
with open('df_closest.pkl', 'wb') as picklefile:
    pickle.dump(df_closest, picklefile)

In [12]:
with open("df_l4_comp.pkl", 'rb') as picklefile: 
    df_l4 = pickle.load(picklefile)

In [13]:
with open("df_all.pkl", 'rb') as picklefile: 
    df_all = pickle.load(picklefile)

In [14]:
df_l4 = df_all.merge(df_l4, how = 'left', on=['INSPECTION_YEAR_Q','BLOCK'])
df_l4.fillna(0, inplace=True)

In [15]:
for i in range(1,smallest):
    df_closest['closest_' + str(i)] = df_closest['closest_' + str(i)].astype(int)

In [16]:
df9 = df_l4.merge(df_closest, how = 'right', left_on='BLOCK', right_on='Block')
del df9['Block']

In [17]:
for i in range(1,smallest):
    df9 = df9.merge(df_l4, left_on=['INSPECTION_YEAR_Q','closest_' + str(i)], right_on=['INSPECTION_YEAR_Q','BLOCK'])
    del df9['BLOCK_y']
    df9.rename(columns={'last_4_c_tot_y': 'c' + str(i) + '_l4ct', 'last_4_c_yes_y': 'c'+str(i)+'_l4cr','BLOCK_x': 'BLOCK',
                       'last_4_c_tot_x': 'last_4_c_tot','last_4_c_yes_x': 'last_4_c_yes'}, inplace=True)

In [18]:
col_to_keep = ['INSPECTION_YEAR_Q', 'BLOCK', 'last_4_c_tot', 'last_4_c_yes',
               'c1_l4ct', 'c1_l4cr', 'c2_l4ct', 'c2_l4cr', 'c3_l4ct', 'c3_l4cr', 'c4_l4ct', 'c4_l4cr', 
               'c5_l4ct', 'c5_l4cr']
df9 = df9[col_to_keep]

In [19]:
with open('last_4_comp_data.pkl', 'wb') as picklefile:
    pickle.dump(df9, picklefile)